In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate
from models.Encoders import NRMS_Encoder, FIM_Encoder, NPA_Encoder, Pipeline_Encoder, MHA_Encoder, Bert_Encoder, RNN_Encoder, CNN_Encoder
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.SFI import SFI_gating

In [2]:
name='sfi'
hparams = {
    'scale':'demo',
    'mode':'train',
    'device':'cuda:1',
    'epochs':8,
    'batch_size':50,
    'title_size':20,
    'abs_size':40,
    'his_size':50,
    'k':30,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'value_dim':16,
    'query_dim':200,
    'head_num':16,
    'filter_num':150,
    'select':'gating',
    'level':1,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'attrs': ['abs','vert','subvert'],
    # 'attrs': ['title'],
    'save_step':[0],
    'spadam':True,
}
# torch.cuda.set_device(hparams['device'])

In [3]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams)

[2021-04-09 05:46:28,421] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'device': 'cuda:1', 'epochs': 8, 'batch_size': 50, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'k': 30, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'value_dim': 16, 'query_dim': 200, 'head_num': 16, 'filter_num': 150, 'select': 'gating', 'level': 1, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'attrs': ['abs', 'vert', 'subvert'], 'save_step': [0], 'spadam': True}
[2021-04-09 05:46:28,423] INFO (root) preparing dataset...
[2021-04-09 05:46:32,606] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor(hparams['k'], encoder.signal_length)

hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

sfi = SFI_gating(hparams, encoder, interactor).to(hparams['device'])

In [5]:
sfi = train(sfi, hparams, loaders, interval=10)

[2021-04-09 05:46:45,784] INFO (root) training...
epoch 1 , step 50 , loss: 1.5908: : 59it [00:18,  3.26it/s]
[2021-04-09 05:47:06,646] INFO (root) saved model of step 0, epoch 1 at data/model_params/sfi-fim-encoder-gating/demo_epoch1_step0_[hs=50,topk=30].model
epoch 2 , step 50 , loss: 1.5164: : 59it [00:17,  3.39it/s]
[2021-04-09 05:47:26,804] INFO (root) saved model of step 0, epoch 2 at data/model_params/sfi-fim-encoder-gating/demo_epoch2_step0_[hs=50,topk=30].model
epoch 3 , step 50 , loss: 1.4497: : 59it [00:17,  3.39it/s]
[2021-04-09 05:47:46,950] INFO (root) saved model of step 0, epoch 3 at data/model_params/sfi-fim-encoder-gating/demo_epoch3_step0_[hs=50,topk=30].model
epoch 4 , step 50 , loss: 1.3788: : 59it [00:17,  3.39it/s]
[2021-04-09 05:48:07,159] INFO (root) saved model of step 0, epoch 4 at data/model_params/sfi-fim-encoder-gating/demo_epoch4_step0_[hs=50,topk=30].model
epoch 5 , step 50 , loss: 1.3111: : 59it [00:17,  3.42it/s]
[2021-04-09 05:48:27,220] INFO (root) 

In [5]:
evaluate(sfi, hparams, loaders[2], loading=True)

[2021-04-07 08:52:30,442] INFO (root) Loading model from data/model_params/sfi-fim-encoder-gating/demo_epoch8_step0_[hs=50,topk=30].model...
[2021-04-07 08:52:30,445] INFO (root) evaluating...
15062it [02:42, 92.52it/s]
[2021-04-07 08:55:16,852] INFO (root) evaluation results:{'auc': 0.7855, 'mean_mrr': 0.4359, 'ndcg@5': 0.4892, 'ndcg@10': 0.5446, 'epoch': 8, 'step': 0}


{'auc': 0.7855,
 'mean_mrr': 0.4359,
 'ndcg@5': 0.4892,
 'ndcg@10': 0.5446,
 'epoch': 8,
 'step': 0}

In [ ]:
hparams['name'] = '-'.join([name,'nrms',hparams['select']])
nrms_encoder = NRMS_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, nrms_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'npa',hparams['select']])

hparams['user_dim'] = 200
hparams['query_dim'] = 200
hparams['filter_num'] = 400

npa_encoder = NPA_Encoder(hparams, vocab, len(loaders[0].dataset.uid2index))
sfi = SFI_gating(hparams, npa_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'pipeline',hparams['select']])

hparams['pipeline'] = 'sfi-fim'
# hparams['scale'] = 'large'

ppl_encoder = Pipeline_Encoder(hparams)
sfi = SFI_gating(hparams, ppl_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'mha',hparams['select']])

mha_encoder = MHA_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, mha_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'lstm',hparams['select']])
hparams['hidden_dim'] = 256

rnn_encoder = RNN_Encoder(hparams,vocab)
sfi = SFI_gating(hparams, rnn_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'bert',hparams['select']])

hparams['level'] = 2
# hparams['batch_size'] = 10
# hparams['bert'] = 'bert-base-uncased'
hparams['bert'] = 'albert-base-v2'

_, loaders = prepare(hparams)

In [ ]:
bert_encoder = Bert_Encoder(hparams)
sfi = SFI_gating(hparams, bert_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

## The rest is for development